# Datavisualisation Project
#### made by Checchin, Di Salvia, Zamar




### Installing libraries

In [308]:
!pip install dash
!pip install dash_bootstrap_components
!pip install wordcloud
!pip install pycountry

   ---------------------------------------- 0.0/300.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/300.6 kB ? eta -:--:--
   --- ----------------------------------- 30.7/300.6 kB 640.0 kB/s eta 0:00:01
   -------------- ------------------------- 112.6/300.6 kB 1.1 MB/s eta 0:00:01
   ---------------- ----------------------- 122.9/300.6 kB 1.2 MB/s eta 0:00:01
   -------------------- ----------------- 163.8/300.6 kB 893.0 kB/s eta 0:00:01
   -------------------- ----------------- 163.8/300.6 kB 893.0 kB/s eta 0:00:01
   ------------------------- ------------ 204.8/300.6 kB 731.4 kB/s eta 0:00:01
   ------------------------------- ------ 245.8/300.6 kB 752.5 kB/s eta 0:00:01
   -------------------------------------- 300.6/300.6 kB 774.2 kB/s eta 0:00:00


## Loading Database

In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import pycountry
from wordcloud import WordCloud

# Acquiring data
df = pd.read_csv('ds_salaries.csv')
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755 entries, 0 to 3754
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           3755 non-null   int64 
 1   experience_level    3755 non-null   object
 2   employment_type     3755 non-null   object
 3   job_title           3755 non-null   object
 4   salary              3755 non-null   int64 
 5   salary_currency     3755 non-null   object
 6   salary_in_usd       3755 non-null   int64 
 7   employee_residence  3755 non-null   object
 8   remote_ratio        3755 non-null   int64 
 9   company_location    3755 non-null   object
 10  company_size        3755 non-null   object
dtypes: int64(4), object(7)
memory usage: 322.8+ KB


## Parsing

In [2]:
# Funzione per convertire i codici ISO o abbreviazioni in nomi completi di paesi
def country_name_from_code(code):
    try:
        return pycountry.countries.lookup(code).name
    except LookupError:
        return None

In [3]:
# Fix
df['company_location'] = df['company_location'].apply(country_name_from_code)
df['employee_residence'] = df['employee_residence'].apply(country_name_from_code)

df['experience_level'] = df['experience_level'].replace('EN', 'Entry-level')
df['experience_level'] = df['experience_level'].replace('MI', 'Mid-level')
df['experience_level'] = df['experience_level'].replace('SE', 'Senior')
df['experience_level'] = df['experience_level'].replace('EX', 'Executive')

df['employment_type'] = df['employment_type'].replace('PT', 'Part-time')
df['employment_type'] = df['employment_type'].replace('FT', 'Full-time')
df['employment_type'] = df['employment_type'].replace('CT', 'Contract')
df['employment_type'] = df['employment_type'].replace('FL', 'Freelance')

df['company_size'] = df['company_size'].replace('L', "Large")
df['company_size'] = df['company_size'].replace('M', "Medium")
df['company_size'] = df['company_size'].replace('S', "Small")

df['remote_ratio'] = df['remote_ratio'].replace(0,'On-Site')
df['remote_ratio'] = df['remote_ratio'].replace(50,'Half-Remote')
df['remote_ratio'] = df['remote_ratio'].replace(100,'Full-Remote')



df['job_title'] = df['job_title'].apply(lambda x: 'Data Analyst' if x == 'Analyst Data' else x)

## Charts

### question 1, chart 1

In [4]:
# World Cloud

wordcloud = WordCloud(width=1200, height=700, background_color='black', colormap='Blues').generate_from_frequencies(
    df['job_title'].value_counts().to_dict()
)
word_fig = go.Figure(
                data=[go.Image(z=wordcloud.to_array())],
                layout=go.Layout(
                    width=1200,
                    height=700,
                    xaxis_showticklabels=False,
                    yaxis_showticklabels=False,
                    hovermode=False
                ),
            )

### question 1, chart 2

In [5]:
# 1a domanda, grafico 2a
# Calcola i salari medi per ciascun titolo di lavoro e ordina i risultati
df_mean_salary_job_title = df.groupby('job_title')['salary_in_usd'].mean().reset_index().sort_values('salary_in_usd', ascending=False)
fig_mean_salary_job_title_sorted = px.bar(df_mean_salary_job_title,
                                          y='job_title', x='salary_in_usd', orientation='h',
                                          labels={'salary_in_usd': 'Salario Medio (USD)',
                                                  'job_title': 'Titolo di Lavoro'},
                                          title='TOP averange salary',
                                          template="plotly_dark", height=600)

### question 1, chart 3

In [6]:
# Filtraggio dei dati per gli anni 2021, 2022, e 2023
df_filtered = df[df['work_year'].isin([2020,2021, 2022, 2023])]

# Calcolo del salario medio per titolo di lavoro ed esperienza
average_salary_by_job_experience = df_filtered.groupby(['job_title', 'experience_level'])['salary_in_usd'].mean().reset_index()
average_salary_by_job_experience.columns = ['job_title', 'experience_level', 'Salary in $']

# Calcolo del conteggio dei lavori per titolo di lavoro ed esperienza
job_counts = df_filtered.groupby(['job_title', 'experience_level'])['job_title'].count().reset_index(name='Job Count')

# Unione dei conteggi dei lavori con i dati dei salari medi
merged_data = average_salary_by_job_experience.merge(job_counts, on=['job_title', 'experience_level'])

# Ordinamento dei dati per salario in modo decrescente
merged_data_sorted = merged_data.sort_values(by='Salary in $', ascending=False)

# Filtro per considerare solo i titoli di lavoro con un numero minimo di osservazioni (es. almeno 5 osservazioni)
filtered_data = merged_data_sorted[merged_data_sorted['Job Count'] >= 5]

# Visualizzazione dei primi 10 record per controllare i dati filtrati
filtered_data.head(10)

# Creazione del bubble chart utilizzando i dati filtrati
fig_bolle = px.scatter(
    filtered_data,  # Limitazione ai primi 40 record per leggibilità
    x='job_title',
    y='Salary in $',
    size='Job Count',
    color='experience_level',
    template='plotly_dark',
    title='Jobs Distribution from 2020 to 2023 (with Minimum 5 Observations)',
    hover_name='job_title',  # Mostra il titolo del lavoro quando si passa sopra una bolla
    hover_data={'experience_level': True, 'Salary in $': True, 'Job Count': True}  # Dati aggiuntivi su hover
)

fig_bolle.update_layout(
    xaxis=dict(tickangle=45),
    xaxis_title='Job Title',
    yaxis_title='Average Salary in $',
    legend_title='Experience Level'
)

fig_bolle.update_traces(marker=dict(line=dict(width=2, color='DarkSlateGrey')))  # Contorno delle bolle
fig_bolle.update_layout(xaxis={'categoryorder':'total ascending'})  # Ordinamento delle etichette sull'asse x
fig_bolle.update_layout(height=700)  # Dimensioni del grafico
fig_bolle.show()


### question 1, chart 4 is a callback, you will find it below

### question 2, chart 1

In [7]:
# Aggregare i dati per calcolare il salario medio per ogni localizzazione dell'azienda
df_mean_salary_by_location = df.groupby('company_location')['salary_in_usd'].mean().reset_index()
# Creare una mappa interattiva utilizzando Plotly Express
fig_mappa = px.choropleth(df_mean_salary_by_location,
                          locations='company_location',
                          locationmode='country names',
                          color='salary_in_usd',
                          color_continuous_scale=px.colors.sequential.YlGn,
                          labels={'salary_in_usd': 'Averange Salary (USD)'})
fig_mappa.show()

### question 2, chart 2

In [8]:
# Ensure 'work_year' is an integer if it's not already
df['work_year'] = df['work_year'].astype(int)

# Calculate mean and standard deviation of salaries by year and company size
salary_stats_by_year_size = df.groupby(['work_year', 'company_size'])['salary_in_usd'].agg(['mean', 'std']).reset_index()

# Set a factor to reduce the variance indication (e.g., 0.10 for 10% of the standard deviation)
factor = 0.10

# Adjust the upper and lower bounds to reduce the shaded area
salary_stats_by_year_size['upper'] = salary_stats_by_year_size['mean'] + (salary_stats_by_year_size['std'] * factor)
salary_stats_by_year_size['lower'] = salary_stats_by_year_size['mean'] - (salary_stats_by_year_size['std'] * factor)

# Preparing data for plotting with reduced variance
salary_stats_by_year_size.head()

# Set colors for different company sizes for consistency in both plots
colors = ['blue', 'green', 'red']
company_sizes = salary_stats_by_year_size['company_size'].unique()

# Re-initializing the figure to apply the adjusted shaded areas
fig_adjusted_variance = go.Figure()

# Add lines and adjusted shaded areas for each company size with visibility ensured
for i, size in enumerate(company_sizes):
    df_filtered = salary_stats_by_year_size[salary_stats_by_year_size['company_size'] == size]
    # Add the main line for mean salary
    fig_adjusted_variance.add_trace(go.Scatter(x=df_filtered['work_year'], y=df_filtered['mean'], mode='lines+markers',
                    name=size, line=dict(color=colors[i % len(colors)]),
                    hoverinfo='skip'))
    
    # Ensure the shaded area is visible and add it around the line
    fig_adjusted_variance.add_trace(go.Scatter(x=pd.concat([df_filtered['work_year'], df_filtered['work_year'][::-1]]),
                    y=pd.concat([df_filtered['upper'], df_filtered['lower'][::-1]]),
                    fill='toself', fillcolor=colors[i % len(colors)], opacity=0.5,
                    line=dict(color='rgba(255,255,255,0)'), showlegend=False,
                    hoverinfo='skip'))

# Update the layout with customizations for the adjusted variance plot with integer years only
fig_adjusted_variance.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=salary_stats_by_year_size['work_year'].unique(),
        ticktext=[str(year) for year in salary_stats_by_year_size['work_year'].unique()]
    ),
    title="Average Salary Trend Over Years by Company Size with Reduced Variance",
    xaxis_title="Year",
    yaxis_title="Average Salary (USD)",
    plot_bgcolor="#f4f4f4",  # Set background color
    legend_title_text='Company Size',
    legend=dict(
        title_font_size=12,
        font_size=10,
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

# Remove grid lines for a cleaner look in the adjusted variance plot
fig_adjusted_variance.update_xaxes(showgrid=False)
fig_adjusted_variance.update_yaxes(showgrid=False)


# Display the chart with reduced variance indication
fig_adjusted_variance.show()


### question 3, chart 1

In [9]:
# Prepare data for the pie chart
remote_category_counts = df['remote_ratio'].value_counts().reset_index()
remote_category_counts.columns = ['remote_ratio', 'count']

# Create the pie chart for remote work category distribution
fig_pie = px.pie(remote_category_counts, names='remote_ratio', values='count',
                 title="Proportion of Work Modalities",
                 color_discrete_sequence=px.colors.sequential.RdBu,
                category_orders={'remote_ratio': ['Full-Remote', 'On-Site', 'Half-Remote']})

### question 3, chart 2

In [10]:
fig_remote_ratio_salary = px.box(df, x='experience_level', y='salary_in_usd', color='remote_ratio',
                                 labels={'remote_ratio': 'Remote Ratio (%)',
                                         'salary_in_usd': 'Salario (USD)'}, template="xgridoff",
                                category_orders={'experience_level': ['Entry-level', 'Mid-level', 'Senior', 'Executive']},
                                color_discrete_sequence=px.colors.sequential.RdBu)

## HTML app (and question 1, chart 4)

In [11]:
external_stylesheets = [
    {
        'href': 'https://fonts.googleapis.com/css2?family=Bungee+Spice&display=swap',
        'rel': 'stylesheet'
    },
    {
        'href': 'https://fonts.googleapis.com/css2?family=Lilita+One&display=swap',
        'rel': 'stylesheet'
    },
    {
        'href': 'https://fonts.googleapis.com/css2?family=Roboto&display=swap',
        'rel': 'stylesheet'
    },
    'https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css',
    'assets/bootstrap.min.css'
]

app = dash.Dash(external_stylesheets=external_stylesheets, suppress_callback_exceptions=True)

# question 1, chart 2b (i must stay here!!)

@app.callback(
    Output("grafico_esperienza", "figure"),
    Input("filtra_expchart", "value"),
)
def update_experience_chart(filtra_expchart):
    filtered = df.loc[df['job_title'].isin([filtra_expchart])]
    return px.box(filtered, x='experience_level', y='salary_in_usd',
                  color='experience_level',
                  labels={'salary_in_usd': 'Salario (USD)', 'experience_level': 'Livello di Esperienza'},
                  title='Distribuzione dei Salari per Livello di Esperienza', height = 600, template="plotly_dark")


# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

sidebar = html.Div(
    [
        html.H3("Working with Big Data", className="display-6"),
        html.Hr(),
        html.P(
            "Our 3 main questions", className="lead"
        ),
        dbc.Nav(
            [
                dbc.NavLink("Which job should i choose?", href="/", active="exact"),
                dbc.NavLink("Which country or company is more convenient?", href="/page-1", active="exact"),
                dbc.NavLink("What about Smart working...?", href="/page-2", active="exact"),
            ],
            vertical=True,
            pills=True,
        ),
    ],
    style=SIDEBAR_STYLE,
)

content = html.Div(id="page-content", style=CONTENT_STYLE)

app.layout = html.Div([
    dcc.Location(id="url"),
    sidebar,
    content,
    html.Footer(
        children=[
            "Created by Checchin, Di Salvia, Zamar"
        ],
        style={
            'position': 'fixed',
            'bottom': 0,
            'width': '100%',
            'background-color': '#f8f9fa',
            'padding': '10px',
            'text-align': 'center',
            'font-size': '14px',
            'color': '#6c757d'
        }
    )], style={'textAlign': 'center'}
)


@app.callback(Output("page-content", "children"), [Input("url", "pathname")])
def render_page_content(pathname):
    # domanda 1, main
    if pathname == "/":
        return dbc.Container([
            
            # intro graphs + title
            html.Div([
                html.Div([
                    html.H1('Why working with Big Data?',
                        className='mb-3',
                        style={'textAlign': 'center', 'marginTop': '50px', 'font-weight': 'bold', 'font-size': '2.5em'}
                    )
                ]),
                html.Div([
                    html.H6('Most of the people work with Big Data as a...',
                            style={'textAlign': 'center', 'marginTop': '10px', 'font-size': '1em'}
                           ),
                dcc.Graph(figure=word_fig)
                ]),
            ]),
            
            html.Hr(style={'border-top': '4px solid #ccc','color': 'black'}),
            
            # grafico più pagati
            html.Div([
                html.Div([
                    html.H2('Which job is more convenient?',
                            style={'textAlign': 'center', 'marginTop': '100px', 'font-size': '2em'}),
                    html.H6('(in terms of dollar $$$)', style={'textAlign': 'center' , 'marginTop': '10px', 'font-size': '1em'})
                ]),
                html.Div([
                    html.Div(dcc.Graph(figure=fig_mean_salary_job_title_sorted), 
                             style={'marginTop': '40px', 'vertical-align': 'top'}),
                ]),
            ]),
            html.Hr(),
            
            # grafico a bolle
            html.Div([
                html.Div([
                     html.H2('Distribution of most remunerative jobs in Big Data',
                             style={'textAlign': 'center', 'marginTop': '100px', 'font-size': '2em'})
                ]),
                html.Div([
                    dcc.Graph(figure=fig_bolle)
                ], style={'marginTop': '40px', 'vertical-align': 'top'}),
            ]),
            html.Hr(),
            
            # grafico lavori
            html.Div([
                html.Div([
                     html.H2('So... Whitch job you would like to do?',
                             style={'textAlign': 'center', 'marginTop': '100px', 'font-size': '2em'})
                ]),
                html.Div([
                    dcc.Dropdown(
                        id="filtra_expchart",
                        options=[{"label": i, "value": i} for i in df.job_title.unique()],
                        value=df['job_title'][1],
                        clearable=False,
                        style={
                            'color': 'blue',
                            'background-color': 'white',
                            'border-color': '#343a40',
                            'border-width': '3px'
                        }
                    ),
                    dcc.Graph(id="grafico_esperienza"),
                ], style={'marginTop': '40px', 'vertical-align': 'top'} ),
            ], style={'display': 'inline-block', 'vertical-align': 'top'}),
            html.Hr(style={'border-top': '4px solid #ccc','color': 'black'}),
            html.Div([
# CONSIDERAZINI FINALI 1
                html.H5('From this brief introduction we can look at how owr data are distribuited in the dataset and make an idea about the most paied jobs in base of the experience, and we can notice that some level of experience are missing for some jobs',
                             style={'textAlign': 'center', 'marginTop': '150px', 'font-size': '1.5em'})
            ])
        ], id="main-container")
    
    # domanda 2
    elif pathname == "/page-1":
        return dbc.Container([
            
            #title
            html.Div([
                html.H1('Which country fits my interests? ($$$)',
                        className='mb-3', style={'textAlign': 'center', 'marginTop': '100px', 'font-size': '2.5em', 'font-weight': 'bold'})
            ], style={'height': '100px'}),
            
            #mappa
            html.Div([
                html.Div([
                    html.H2('Averange Salaries in the world',
                            style={'textAlign': 'center', 'marginTop': '20px', 'font-size': '2em'}
                           )
                ], style={'height': '50px'}),
                dcc.Graph(figure=fig_mappa)
            ]),
            html.Hr(),
            
            #aziende
            html.Div([
                html.Div([
                    html.H2('Andament of Salaries by size of Companies',
                            style={'textAlign': 'center', 'marginTop': '20px', 'font-size': '2em'}
                           )
                ], style={'height': '50px'}),
                dcc.Graph(figure=fig_adjusted_variance)
            ], style={'marginTop': '100px'}),
            html.Hr(style={'border-top': '4px solid #ccc','color': 'black'}),
            html.Div([
# CONSIDERAZINI FINALI 2
                html.H5('From the first chart we can notice that israel is paying the most, also Puerto Rico, USA, Russia, Canada and New Zeland. From the second we can see that the Medium Size Company are paying their employers much more than in 2021, Small Company are decresing salaries from 2021, and the Lrge ones are constantly increasing',
                             style={'textAlign': 'center', 'marginTop': '150px', 'font-size': '1.5em'})
            ])
        ], id="main2-container")
    
    #domanda 3, end
    elif pathname == "/page-2":
        return dbc.Container([
            
            #title
            html.Div([
                html.H1('Smart Working',
                        className='mb-3',
                        style={'textAlign': 'center', 'marginTop': '100px', 'font-size': '2.5em', 'font-weight': 'bold'})
            ], style={'height': '100px'}),
            
            # torta
            html.Div([
                dcc.Graph(figure=fig_pie)
            ], style={'marginLeft': '50px'}),
            
            # barplot
            html.Div([
                html.H2('Andament of Salaries by size of Companies',
                            style={'textAlign': 'center', 'marginTop': '20px', 'font-size': '2em'}
                           )
                ], style={'height': '50px'}),
            html.Div([
                dcc.Graph(figure=fig_remote_ratio_salary)
            ], style={'marginLeft': '50px'}),
            html.Hr(style={'border-top': '4px solid #ccc','color': 'black'}),
            html.Div([
# CONSIDERAZINI FINALI 3
                html.H5('The situation of the works in our dataset shows more only On-Site or Full-Remote Worker than Half-Remote Worker, also we can notice that Half-Remote Worker are usually payed less. Full-Remote Working also is payed less than On-Site if the experience level is low, more if the experience level is high',
                             style={'textAlign': 'center', 'marginTop': '150px', 'font-size': '1.5em'})
            ])
        ], id="main3-container")
    
    # If the user tries to reach a different page, return a 404 message
    return html.Div(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ],
        className="p-3 bg-light rounded-3",
    )
# Esegui l'app
if __name__ == '__main__':
    app.run_server(debug=True)